In [8]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from transformers import BertTokenizer, BertForSequenceClassification
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import accuracy_score, hamming_loss
import torch
from transformers import BertTokenizer, BertModel
from skmultilearn.adapt import MLkNN
from nltk.tokenize import TweetTokenizer
import string
import re
import emoji

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
 

/kaggle/input/hatespeech/train.csv
/kaggle/input/hate-speech-detection/sample_submission.csv
/kaggle/input/hate-speech-detection/train.csv
/kaggle/input/hate-speech-detection/test.csv


In [14]:
df=pd.read_csv("/kaggle/input/hatespeech/train.csv")
test=pd.read_csv("/kaggle/input/hate-speech-detection/test.csv")

In [15]:
emoticons = [':-)', ':)', '(:', '(-:', ':))', '((:', ':-D', ':D', 'X-D', 'XD', 'xD', 'xD', '<3', '</3', ':\*',
                 ';-)',
                 ';)', ';-D', ';D', '(;', '(-;', ':-(', ':(', '(:', '(-:', ':,(', ':\'(', ':"(', ':((', ':D', '=D',
                 '=)',
                 '(=', '=(', ')=', '=-O', 'O-=', ':o', 'o:', 'O:', 'O:', ':-o', 'o-:', ':P', ':p', ':S', ':s', ':@',
                 ':>',
                 ':<', '^_^', '^.^', '>.>', 'T_T', 'T-T', '-.-', '*.*', '~.~', ':*', ':-*', 'xP', 'XP', 'XP', 'Xp',
                 ':-|',
                 ':->', ':-<', '$_$', '8-)', ':-P', ':-p', '=P', '=p', ':*)', '*-*', 'B-)', 'O.o', 'X-(', ')-X']

def tokenize(tweet):
    # instantiate the tokenizer class
    tokenizer = TweetTokenizer(preserve_case=False, 
                              strip_handles=True,
                              reduce_len=True)

    # tokenize the tweets
    tweet_tokens = tokenizer.tokenize(tweet)

    tweets_clean = []
    for word in tweet_tokens: # Go through every word in your tokens list
        if word not in string.punctuation:  # remove punctuation
            tweets_clean.append(word)
    result = tweets_clean
    return " ".join(result)

def preprocess(tweet):
    result = tweet.replace('rt @','@')
    result = result.replace('@','<user> @')
    # it will remove hyperlinks
    result = re.sub(r'https?:\/\/.*[\r\n]*', '<url>', result)

    # it will remove hashtags. We have to be careful here not to remove 
    # the whole hashtag because text of hashtags contains huge information. 
    # only removing the hash # sign from the word
    result = re.sub(r'#', '<hashtag>', result)

    # Replace multiple dots with space
    result = re.sub('\.\.+', ' ', result) 

    for char in result:
        if emoji.is_emoji(char):
            result = result.replace(char, "<emoticon >")
    for emo in emoticons:
        result = result.replace(emo, "<emoticon >")

    result = tokenize(result)
    # it will remove single numeric terms in the tweet. 
    result = re.sub(r'[0-9]+', '<number>', result)
    result = re.sub(r'<number>\s?st', '<number>', result)
    result = re.sub(r'<number>\s?nd', '<number>', result)
    result = re.sub(r'<number>\s?rd', '<number>', result)
    result = re.sub(r'<number>\s?th', '<number>', result)

    return result

def clean_abstract(text):
    # Apply the preprocessing steps to the provided text
    result = preprocess(text.lower())
    return result

In [16]:
df['text'] = df['text'].apply(clean_abstract)

# Print the first few rows to verify the changes
print(df.head())

     id                                               text  HS  TR  AG
0  6452  <user> you're the exception you weren't a rude...   1   1   0
1  4884  if a woman doesn't want you just unleash your ...   1   0   0
2  1931  son of jamestown protestants that made the usa...   0   0   0
3  4942  literally just got hit by a car bc this dumb b...   1   1   0
4  4721  charli fuck you bitch charli omg why am i so e...   1   1   0


In [17]:
from transformers import BertTokenizer, BertModel
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer
from skmultilearn.adapt import MLkNN
from lightgbm import LGBMClassifier
import torch

# # Load pre-trained BERTweet tokenizer and model
# tokenizer = BertTokenizer.from_pretrained('vinai/bertweet-base')
# model = BertModel.from_pretrained('vinai/bertweet-base')


In [18]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from skmultilearn.problem_transform import BinaryRelevance
from sklearn.metrics import accuracy_score, hamming_loss
from lightgbm import LGBMClassifier

# Assuming df is your DataFrame containing features and labels
# Replace X with your features and y with your labels
X_text = df['text'].tolist()  # Assuming 'text_column' contains the text data
y=df[["HS", 'TR', 'AG']]  # Assuming 'label1', 'label2', 'label3' are your multilabels

tfidf_vectorizer = TfidfVectorizer(max_features=10000)  # You can adjust max_features as needed
X_tfidf = tfidf_vectorizer.fit_transform(X_text)

# Step 2: Split the data into train and validation sets (80% train, 20% validation)
X_train, X_val, y_train, y_val = train_test_split(X_tfidf, y, test_size=0.2, random_state=42)

In [19]:
print(y_train.shape)

(4799, 3)


In [20]:
from sklearn.model_selection import KFold
import numpy as np

# Define the number of folds for cross-validation
num_folds = 5

# Initialize lists to store evaluation metrics for each fold
accuracies = []
hamming_losses = []

# Perform k-fold cross-validation
kf = KFold(n_splits=num_folds, shuffle=True, random_state=42)
for train_index, val_index in kf.split(X_tfidf, y):
    # Split the data into train and validation sets for this fold
    X_train_fold, X_val_fold = X_tfidf[train_index], X_tfidf[val_index]
    y_train_fold, y_val_fold = y.iloc[train_index], y.iloc[val_index]

    # Train Binary Relevance classifier with LightGBM
    classifier = BinaryRelevance(classifier=LGBMClassifier(), require_dense=[False, True])
    classifier.fit(X_train_fold, y_train_fold)

    # Predict on validation set
    predictions = classifier.predict(X_val_fold)

    # Evaluate the model
    accuracy = accuracy_score(y_val_fold, predictions)
    hamming_loss_val = hamming_loss(y_val_fold, predictions)

    # Store evaluation metrics for this fold
    accuracies.append(accuracy)
    hamming_losses.append(hamming_loss_val)

# Calculate average evaluation metrics across all folds
avg_accuracy = np.mean(accuracies)
avg_hamming_loss = np.mean(hamming_losses)

print("Average Accuracy across all folds:", avg_accuracy)
print("Average Hamming Loss across all folds:", avg_hamming_loss)


[LightGBM] [Info] Number of positive: 2055, number of negative: 2744
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.014671 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 19041
[LightGBM] [Info] Number of data points in the train set: 4799, number of used features: 555
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.428214 -> initscore=-0.289141
[LightGBM] [Info] Start training from score -0.289141
[LightGBM] [Info] Number of positive: 745, number of negative: 4054
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.011815 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 19041
[LightGBM] [Info] Number of data points in the train set: 4799, number of used features: 555
[LightGBM] [Info] [bi

In [22]:
X_test_data = test['text'].tolist()
X_test_tfidf = tfidf_vectorizer.transform(X_test_data)  # Use transform instead of fit_transform
predictions_test = classifier.predict(X_test_tfidf)


In [18]:
print(predictions_test)

  (7, 0)	1

  (8, 0)	1

  (11, 0)	1

  (13, 0)	1

  (14, 0)	1

  (15, 0)	1

  (17, 0)	1

  (19, 0)	1

  (24, 0)	1

  (26, 0)	1

  (29, 0)	1

  (30, 0)	1

  (31, 0)	1

  (33, 0)	1

  (34, 0)	1

  (35, 0)	1

  (37, 0)	1

  (41, 0)	1

  (42, 0)	1

  (44, 0)	1

  (46, 0)	1

  (47, 0)	1

  (48, 0)	1

  (50, 0)	1

  (53, 0)	1

  :	:

  (1803, 2)	1

  (1807, 2)	1

  (1828, 2)	1

  (1831, 2)	1

  (1837, 2)	1

  (1845, 2)	1

  (1849, 2)	1

  (1852, 2)	1

  (1857, 2)	1

  (1871, 2)	1

  (1875, 2)	1

  (1877, 2)	1

  (1884, 2)	1

  (1885, 2)	1

  (1886, 2)	1

  (1898, 2)	1

  (1919, 2)	1

  (1932, 2)	1

  (1936, 2)	1

  (1938, 2)	1

  (1984, 2)	1

  (1987, 2)	1

  (1989, 2)	1

  (1991, 2)	1

  (1995, 2)	1


In [23]:
predictions_df = pd.DataFrame(predictions_test.toarray(), columns=y.columns)

In [24]:
predictions_df['id'] = test["id"].values

In [25]:
predictions_df.to_csv("submission.csv",index=False)